In [1]:
import pandas as pd
from transformers import LongformerTokenizerFast, LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig

import torch.nn as nn
import torch
import numpy as np
import os
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from preprocess import preprocess_data, get_dataset_and_labels
from sklearn.model_selection import train_test_split

In [2]:
max_sequence_length = 256
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = max_sequence_length)
model = LongformerForSequenceClassification.from_pretrained('./results/twitter-rumour-classification/', local_files_only=True).to("cuda")

In [17]:
def convert_label(label):
    if label == "rumour":
        return 1
    elif label == "non-rumour":
        return 0
    else:
        raise Exception("label classes must be 'rumour' or 'non-rumour'")


def convert_label_to_rumour(label):
    if label == 1:
        return "rumour"
    elif label == 0:
        return "non-rumour"
    else:
        raise Exception("label classes must be 1 or 0")


def get_labels(label_path, sourceIds):
    with open(label_path) as f:
        labels = json.load(f)
    corresponding_labels = [labels[id] for id in sourceIds]
    numeric_labels = [convert_label(label) for label in corresponding_labels]

    return numeric_labels

In [4]:
test_path = "./project_data/test.data.jsonl"
test_texts, sourceIds = preprocess_data(data_path=test_path, max_sequence_length=max_sequence_length) 

In [5]:
test_encodings = tokenizer(test_texts, padding = 'max_length', truncation=True, max_length = max_sequence_length)

In [8]:
class TestDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.tokenized_texts.items()}

test_dataset = TestDataset(test_encodings)

In [9]:
from transformers import default_data_collator

label_ids: torch.Tensor = None
preds: torch.Tensor = None

with torch.no_grad():
    dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

    for batch in tqdm(dataloader):

        batch['input_ids'] = batch['input_ids'].cuda()
            
        predictions = model(input_ids=batch['input_ids']
                                   )
        
        predictions = predictions[0]

        if preds is None:
            preds = predictions.detach().sigmoid()
        else:
            preds = torch.cat((preds, predictions.detach()), dim=0)


        # if label_ids is None:
        #     label_ids = batch["labels"].detach()
        # else:
        #     label_ids = torch.cat((label_ids, batch["labels"].detach()), dim=0)
        

100%|██████████| 73/73 [00:11<00:00,  6.59it/s]


In [11]:
predictions = np.argmax(preds.to("cpu"), axis=1)

In [18]:
predictions_dict = {sourceId: convert_label_to_rumour(prediction) for sourceId, prediction in zip(sourceIds, predictions)}

In [23]:
with open("test-output.json", "w") as outputfile:
    json.dump(predictions_dict, outputfile)

In [25]:
data_val_path = './project_data/dev.data.jsonl'
val_labels_path = './project_data/dev.label.json'

val_texts, val_sourceIds = preprocess_data(data_path=data_val_path, max_sequence_length=max_sequence_length)
val_labels = get_labels(val_labels_path, val_sourceIds)
val_encodings = tokenizer(val_texts, padding = 'max_length', truncation=True, max_length = max_sequence_length)
val_dataset = TestDataset(val_encodings)

In [26]:
label_ids: torch.Tensor = None
val_preds: torch.Tensor = None

with torch.no_grad():
    dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

    for batch in tqdm(dataloader):

        batch['input_ids'] = batch['input_ids'].cuda()
            
        val_predictions = model(input_ids=batch['input_ids']
                                   )
        
        val_predictions = val_predictions[0]

        if val_preds is None:
            val_preds = val_predictions.detach().sigmoid()
        else:
            val_preds = torch.cat((val_preds, val_predictions.detach()), dim=0)


100%|██████████| 73/73 [00:10<00:00,  6.95it/s]


In [28]:


from sklearn.metrics import precision_recall_fscore_support
val_predictions = np.argmax(val_preds.to("cpu"), axis=1)
p, r, f, _ = precision_recall_fscore_support(val_predictions, val_labels, pos_label=1, average="binary")

In [29]:
print("Precision: ", p)
print("Recall: ", r)
print("F1: ", f)

Precision:  0.7967914438502673
Recall:  0.8713450292397661
F1:  0.8324022346368715
